In [4]:
import os
import mindspore as ms
import mindspore.nn as nn
from mindspore import context, Tensor
from mindspore.common import set_seed
from mindspore.train.model import Model
from mindspore.train.callback import TimeMonitor, LossMonitor, CheckpointConfig, ModelCheckpoint
from mindspore.communication.management import init, get_rank, get_group_size
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from dataset import create_train_dataset
from srcnn import SRCNN
from model_utils.config import config

set_seed(1)

def filter_checkpoint_parameter_by_list(origin_dict, param_filter):
    """remove useless parameters according to filter_list"""
    for key in list(origin_dict.keys()):
        for name in param_filter:
            if name in key:
                print("Delete parameter from checkpoint: ", key)
                del origin_dict[key]
                break


def run_train():
    cfg = config
    data_path = cfg.data_path
    pretrained_ckpt_path = cfg.pre_trained_path
    data_path += "/srcnn.mindrecord00"
    output_path = cfg.output_path
    if cfg.device_target == "CPU":
        print("Running on CPU")
        if cfg.run_distribute:
            init()
        context.set_context(mode=context.GRAPH_MODE,
                            device_target=cfg.device_target,
                            save_graphs=False)
    else:
        raise ValueError("Unsupported device target.")

    rank = 0
    device_num = 1
    train_dataset = create_train_dataset(data_path, batch_size=cfg.batch_size,
                                         shard_id=rank, num_shard=device_num)
    step_size = train_dataset.get_dataset_size()

    # define net
    net = SRCNN()

    # init weight
    if cfg.pre_trained_path:
        param_dict = load_checkpoint(pretrained_ckpt_path)
        if cfg.filter_weight:
            filter_list = [x.name for x in net.end_point.get_parameters()]
            filter_checkpoint_parameter_by_list(param_dict, filter_list)
        load_param_into_net(net, param_dict)

    lr = Tensor(float(cfg.lr), ms.float32)

    opt = nn.Adam(params=net.trainable_params(), learning_rate=lr, eps=1e-07)
    loss = nn.MSELoss(reduction='mean')
    model = Model(net, loss_fn=loss, optimizer=opt)

    # define callbacks
    callbacks = [LossMonitor(), TimeMonitor(data_size=step_size)]
    if cfg.save_checkpoint and rank == 0:
        config_ck = CheckpointConfig(save_checkpoint_steps=step_size,
                                     keep_checkpoint_max=cfg.keep_checkpoint_max)
        save_ckpt_path = os.path.join(output_path, 'ckpt_' + str(rank) + '/')
        ckpt_cb = ModelCheckpoint(prefix="srcnn", directory=save_ckpt_path, config=config_ck)
        callbacks.append(ckpt_cb)

    model.train(cfg.epoch_size, train_dataset, callbacks=callbacks, dataset_sink_mode=True)

if __name__ == '__main__':
    run_train()


{'enable_modelarts': 'Whether training on modelarts, default: False', 'data_url': 'Url for modelarts', 'train_url': 'Url for modelarts', 'data_path': 'The location of the input data.', 'output_path': 'The location of the output file.', 'device_target': 'Device target, support GPU and Ascend.', 'enable_profiling': 'Whether enable profiling while training, default: False'}
{'batch_size': 8,
 'checkpoint_path': '/cache/checkpoint_path.ckpt',
 'ckpt_file': '',
 'config_path': 'G:\\PythonProject\\Peter\\model_utils\\../config.yaml',
 'data_path': 'dataset/mindrecord',
 'device_num': 1,
 'device_target': 'CPU',
 'enable_profiling': False,
 'epoch_size': 40,
 'file_format': 'MINDIR',
 'file_name': 'srcnn',
 'filter_weight': False,
 'image_height': 512,
 'image_path': '',
 'image_width': 512,
 'keep_checkpoint_max': 20,
 'lr': 0.0001,
 'output_folder': 'dataset/mindrecord',
 'output_path': 'output/train',
 'patch_size': 33,
 'pre_trained_path': '',
 'predict_path': '',
 'pretrained_ckpt_path':

epoch: 1 step: 145, loss is 0.007453128695487976
epoch: 1 step: 146, loss is 0.0031996332108974457
epoch: 1 step: 147, loss is 0.0036160338204354048
epoch: 1 step: 148, loss is 0.002368620363995433
epoch: 1 step: 149, loss is 0.002386759500950575
epoch: 1 step: 150, loss is 0.002343876985833049
epoch: 1 step: 151, loss is 0.006198520306497812
epoch: 1 step: 152, loss is 0.004257000517100096
epoch: 1 step: 153, loss is 0.0038512800820171833
epoch: 1 step: 154, loss is 0.002820122754201293
epoch: 1 step: 155, loss is 0.006789589766412973
epoch: 1 step: 156, loss is 0.0023760416079312563
epoch: 1 step: 157, loss is 0.004681270103901625
epoch: 1 step: 158, loss is 0.0040529766120016575
epoch: 1 step: 159, loss is 0.0038051640149205923
epoch: 1 step: 160, loss is 0.0018088299548253417
epoch: 1 step: 161, loss is 0.001459497376345098
epoch: 1 step: 162, loss is 0.004867487587034702
epoch: 1 step: 163, loss is 0.0029953056946396828
epoch: 1 step: 164, loss is 0.0043536582961678505
epoch: 1 st

epoch: 1 step: 311, loss is 0.003285511862486601
epoch: 1 step: 312, loss is 0.0022621904499828815
epoch: 1 step: 313, loss is 0.0017937386874109507
epoch: 1 step: 314, loss is 0.002254137070849538
epoch: 1 step: 315, loss is 0.0034013206604868174
epoch: 1 step: 316, loss is 0.0017923510167747736
epoch: 1 step: 317, loss is 0.0015948433429002762
epoch: 1 step: 318, loss is 0.0013446728698909283
epoch: 1 step: 319, loss is 0.0016964789247140288
epoch: 1 step: 320, loss is 0.0010923987720161676
epoch: 1 step: 321, loss is 0.0032983100973069668
epoch: 1 step: 322, loss is 0.003750690259039402
epoch: 1 step: 323, loss is 0.0011346094543114305
epoch: 1 step: 324, loss is 0.002638805890455842
epoch: 1 step: 325, loss is 0.0024413757491856813
epoch: 1 step: 326, loss is 0.0016538111958652735
epoch: 1 step: 327, loss is 0.0010264447191730142
epoch: 1 step: 328, loss is 0.0015991705004125834
epoch: 1 step: 329, loss is 0.0019310836214572191
epoch: 1 step: 330, loss is 0.0026087630540132523
epoc

epoch: 2 step: 102, loss is 0.001379743916913867
epoch: 2 step: 103, loss is 0.0009414719534106553
epoch: 2 step: 104, loss is 0.002698000753298402
epoch: 2 step: 105, loss is 0.0021875936072319746
epoch: 2 step: 106, loss is 0.0021468179766088724
epoch: 2 step: 107, loss is 0.0015071795787662268
epoch: 2 step: 108, loss is 0.0013053730363026261
epoch: 2 step: 109, loss is 0.00038927618879824877
epoch: 2 step: 110, loss is 0.0010850675171241164
epoch: 2 step: 111, loss is 0.0009068789076991379
epoch: 2 step: 112, loss is 0.0017049222951754928
epoch: 2 step: 113, loss is 0.00034177687484771013
epoch: 2 step: 114, loss is 0.001077385968528688
epoch: 2 step: 115, loss is 0.001534982118755579
epoch: 2 step: 116, loss is 0.0013157689245417714
epoch: 2 step: 117, loss is 0.0017168825725093484
epoch: 2 step: 118, loss is 0.001645573996938765
epoch: 2 step: 119, loss is 0.0017343725776299834
epoch: 2 step: 120, loss is 0.0013433349085971713
epoch: 2 step: 121, loss is 0.000996672548353672
epoc

epoch: 2 step: 267, loss is 0.0012189419940114021
epoch: 2 step: 268, loss is 0.0005740377237088978
epoch: 2 step: 269, loss is 0.0011942500714212656
epoch: 2 step: 270, loss is 0.001340020215138793
epoch: 2 step: 271, loss is 0.001871513552032411
epoch: 2 step: 272, loss is 0.0014229284133762121
epoch: 2 step: 273, loss is 0.0012804941507056355
epoch: 2 step: 274, loss is 0.001413974561728537
epoch: 2 step: 275, loss is 0.0010745954932644963
epoch: 2 step: 276, loss is 0.0008105825399979949
epoch: 2 step: 277, loss is 0.0005032553453929722
epoch: 2 step: 278, loss is 0.0008722117054276168
epoch: 2 step: 279, loss is 0.0008864098344929516
epoch: 2 step: 280, loss is 0.001196652534417808
epoch: 2 step: 281, loss is 0.0017174968961626291
epoch: 2 step: 282, loss is 0.0009581856429576874
epoch: 2 step: 283, loss is 0.0017402766970917583
epoch: 2 step: 284, loss is 0.0007569561130367219
epoch: 2 step: 285, loss is 0.0008139471174217761
epoch: 2 step: 286, loss is 0.0005760536296293139
epoc

...

epoch: 39 step: 11, loss is 0.0009777171071618795
epoch: 39 step: 12, loss is 0.0005298706819303334
epoch: 39 step: 13, loss is 0.000705576385371387
epoch: 39 step: 14, loss is 0.0010161349782720208
epoch: 39 step: 15, loss is 0.0005179695435799658
epoch: 39 step: 16, loss is 0.001776419230736792
epoch: 39 step: 17, loss is 0.00028986658435314894
epoch: 39 step: 18, loss is 0.00047427459503524005
epoch: 39 step: 19, loss is 0.0007862251368351281
epoch: 39 step: 20, loss is 0.0012601593043655157
epoch: 39 step: 21, loss is 0.000287965580355376
epoch: 39 step: 22, loss is 0.0003997463791165501
epoch: 39 step: 23, loss is 0.0005011296598240733
epoch: 39 step: 24, loss is 0.0007346021011471748
epoch: 39 step: 25, loss is 0.0001716547558316961
epoch: 39 step: 26, loss is 0.00039240613114088774
epoch: 39 step: 27, loss is 0.0009746482828631997
epoch: 39 step: 28, loss is 0.0002424360136501491
epoch: 39 step: 29, loss is 0.0006136837182566524
epoch: 39 step: 30, loss is 0.00066309206886217
ep

epoch: 39 step: 174, loss is 0.00043647675192914903
epoch: 39 step: 175, loss is 0.0003392854705452919
epoch: 39 step: 176, loss is 0.0004933039890602231
epoch: 39 step: 177, loss is 0.0007268933695740998
epoch: 39 step: 178, loss is 0.00087397126480937
epoch: 39 step: 179, loss is 0.0006442226585932076
epoch: 39 step: 180, loss is 0.001988364616408944
epoch: 39 step: 181, loss is 0.00011947137682000175
epoch: 39 step: 182, loss is 0.0006235427572391927
epoch: 39 step: 183, loss is 0.0001372279948554933
epoch: 39 step: 184, loss is 0.0004956359043717384
epoch: 39 step: 185, loss is 0.0002987765474244952
epoch: 39 step: 186, loss is 0.0010293324012309313
epoch: 39 step: 187, loss is 0.0010734967654570937
epoch: 39 step: 188, loss is 0.000594506913330406
epoch: 39 step: 189, loss is 0.00029987958259880543
epoch: 39 step: 190, loss is 0.00022453466954175383
epoch: 39 step: 191, loss is 0.001449767267331481
epoch: 39 step: 192, loss is 0.0008248287485912442
epoch: 39 step: 193, loss is 0.0

epoch: 39 step: 335, loss is 0.0006362419226206839
epoch: 39 step: 336, loss is 0.0014085170114412904
epoch: 39 step: 337, loss is 0.0011733720311895013
epoch: 39 step: 338, loss is 0.00024533073883503675
epoch: 39 step: 339, loss is 0.0004909445415250957
epoch: 39 step: 340, loss is 0.0007626772276125848
epoch: 39 step: 341, loss is 0.0009564577485434711
epoch: 39 step: 342, loss is 0.0005950136692263186
epoch: 39 step: 343, loss is 0.00048611024976707995
epoch: 39 step: 344, loss is 0.0003772411437239498
epoch: 39 step: 345, loss is 0.0005149916396476328
epoch: 39 step: 346, loss is 0.0007559137884527445
epoch: 39 step: 347, loss is 0.0003116715815849602
epoch: 39 step: 348, loss is 0.00041198235703632236
epoch: 39 step: 349, loss is 0.00041377567686140537
epoch: 39 step: 350, loss is 0.0006058310973457992
epoch: 39 step: 351, loss is 0.0004016759339720011
epoch: 39 step: 352, loss is 0.0004176636866759509
epoch: 39 step: 353, loss is 0.0005648788646794856
epoch: 39 step: 354, loss i

epoch: 40 step: 122, loss is 0.0010575447231531143
epoch: 40 step: 123, loss is 0.0006525187636725605
epoch: 40 step: 124, loss is 0.0012336832005530596
epoch: 40 step: 125, loss is 0.0011622821912169456
epoch: 40 step: 126, loss is 0.0012394771911203861
epoch: 40 step: 127, loss is 0.0014913321938365698
epoch: 40 step: 128, loss is 0.0008429442532360554
epoch: 40 step: 129, loss is 0.0005997297121211886
epoch: 40 step: 130, loss is 0.0007755306432954967
epoch: 40 step: 131, loss is 0.0013261476997286081
epoch: 40 step: 132, loss is 0.0007262879516929388
epoch: 40 step: 133, loss is 0.00047773943515494466
epoch: 40 step: 134, loss is 0.000564312853384763
epoch: 40 step: 135, loss is 0.0005122387665323913
epoch: 40 step: 136, loss is 0.001319135189987719
epoch: 40 step: 137, loss is 0.000541942601557821
epoch: 40 step: 138, loss is 0.0006543431081809103
epoch: 40 step: 139, loss is 0.0006592058343812823
epoch: 40 step: 140, loss is 0.0003629135899245739
epoch: 40 step: 141, loss is 0.00

epoch: 40 step: 283, loss is 0.0008198383147828281
epoch: 40 step: 284, loss is 0.001320548471994698
epoch: 40 step: 285, loss is 0.0003405412717256695
epoch: 40 step: 286, loss is 0.0002591308730188757
epoch: 40 step: 287, loss is 0.0009371421183459461
epoch: 40 step: 288, loss is 0.0008898705709725618
epoch: 40 step: 289, loss is 0.0007663614233024418
epoch: 40 step: 290, loss is 0.0002824093389790505
epoch: 40 step: 291, loss is 0.0004151395696680993
epoch: 40 step: 292, loss is 0.0008400582009926438
epoch: 40 step: 293, loss is 0.00041232569492422044
epoch: 40 step: 294, loss is 0.00028259167447686195
epoch: 40 step: 295, loss is 0.0015431276988238096
epoch: 40 step: 296, loss is 0.0003662792732939124
epoch: 40 step: 297, loss is 0.0004712776280939579
epoch: 40 step: 298, loss is 0.000884348526597023
epoch: 40 step: 299, loss is 0.0007622975390404463
epoch: 40 step: 300, loss is 0.0012349358294159174
epoch: 40 step: 301, loss is 0.00030859961407259107
epoch: 40 step: 302, loss is 0